In [1]:
import sys,os
add_path = os.path.abspath('../python')
if not add_path in sys.path:
    sys.path.append(add_path)
print(sys.path.index(add_path))

15


In [2]:
import time
import numpy as np
from numpy import vstack, hstack
from segbench import csaAssign, correspondPixels,correspondVoxels, matchEdgeMaps2D, matchEdgeMaps3D

In [3]:
data = np.array([[1, 4, 3],
    [2, 5, 3],
    [3, 6, 3],
    [1, 5, 1],
    [2, 6, 1],
    [3, 4, 5]])
n = 6
e1 = csaAssign(n,data)
print e1

[[ 1.  5.  1.]
 [ 2.  6.  1.]
 [ 3.  4.  5.]]


In [4]:
# Unit test for csaAssign.
n = 1000
dg = np.random.rand(n,n)>.5
m = np.sum(dg.flat)
i = np.array(dg.ravel().nonzero()).astype(float)
g1 = 1+np.floor(np.divide(i,n))
g2 = (1+np.floor(np.divide(i,n)),
     1+ np.mod(i,n)+n,
     1+np.floor(np.random.rand(1,m))*1000)
g2 = np.array(np.vstack(g2)).T
t = time.time()
e2 = csaAssign(n*2,g2)
print '{:.3e}s'.format(time.time() - t)
esum = np.sum(e2,axis=0)
if not all(esum[0:2]==(n*(n+1)/2,n*(n+1)/2+n*n)): raise ValueError("bug")
if not np.sum(esum[0:2])==(2*n*(2*n+1)/2): raise ValueError("bug")
print n,m, e2[3][:]

5.577e-02s
1000 500161 [  4.00000000e+00   1.00500000e+03   1.00000000e+00]


In [5]:
# Unit test for correspondPixels.
for n in np.round(np.logspace(0,2,10)).astype(int):
    for thresh in np.linspace(0,1,10):
        print 'n={:d} thresh={:1.2e}'.format(n,thresh),
        bmap1 = (np.random.rand(n,n)>thresh).astype(float)
        bmap2 = (np.random.rand(n,n)>thresh).astype(float)
        t = time.time()
        cost,oc,match1,match2 = correspondPixels(bmap1,bmap2,.005)
        print '{:1.2e}s'.format(time.time() - t),
        npix = len(bmap1.nonzero()[0]) + len(bmap2.nonzero()[0])
        ncor = len(match1.nonzero()[0]) + len(match1.nonzero()[0])
        nout = len(((bmap1>0)&(match1==0)).nonzero()[0]) + len(((bmap2>0)&(match2==0)).nonzero()[0])
        print npix,ncor,nout
        if ncor+nout!=npix: raise ValueError("bug")

        match1f = match1.ravel()
        idx1 = match1f.astype(int)
        a1 = match1f.nonzero()[0]
        a2 = match1f[idx1.nonzero()[0]].ravel()
        a = vstack((a1,a2)).T

        match2f = match2.ravel()
        idx2 = match2f.astype(int)
        b1 = match2f.nonzero()[0]
        b2 = match2f[idx2.nonzero()[0]].ravel()
        b = vstack((b2,b1)).T
        b = b[b[:,0].argsort(),]
        if np.sum(a-b): raise ValueError("bug")

n=1 thresh=0.00e+00 8.70e-05s 2 0 2
n=1 thresh=1.11e-01 4.98e-05s 2 0 2
n=1 thresh=2.22e-01 4.29e-05s 2 0 2
n=1 thresh=3.33e-01 2.50e-05s 1 0 1
n=1 thresh=4.44e-01 3.89e-05s 2 0 2
n=1 thresh=5.56e-01 2.60e-05s 1 0 1
n=1 thresh=6.67e-01 3.19e-05s 0 0 0
n=1 thresh=7.78e-01 2.41e-05s 0 0 0
n=1 thresh=8.89e-01 2.29e-05s 0 0 0
n=1 thresh=1.00e+00 2.29e-05s 0 0 0
n=2 thresh=0.00e+00 7.70e-05s 8 6 2
n=2 thresh=1.11e-01 7.70e-05s 8 6 2
n=2 thresh=2.22e-01 6.29e-05s 5 2 3
n=2 thresh=3.33e-01 5.79e-05s 6 2 4
n=2 thresh=4.44e-01 4.32e-05s 4 2 2
n=2 thresh=5.56e-01 4.10e-05s 5 2 3
n=2 thresh=6.67e-01 2.72e-05s 1 0 1
n=2 thresh=7.78e-01 2.50e-05s 2 0 2
n=2 thresh=8.89e-01 2.60e-05s 2 0 2
n=2 thresh=1.00e+00 2.31e-05s 0 0 0
n=3 thresh=0.00e+00 1.74e-04s 18 16 2
n=3 thresh=1.11e-01 1.59e-04s 16 12 4
n=3 thresh=2.22e-01 9.51e-05s 12 10 2
n=3 thresh=3.33e-01 7.70e-05s 12 8 4
n=3 thresh=4.44e-01 4.82e-05s 7 2 5
n=3 thresh=5.56e-01 4.60e-05s 8 4 4
n=3 thresh=6.67e-01 4.10e-05s 6 2 4
n=3 thresh=7.78e-01 2

In [6]:
# Unit test for matchEdgeMaps2D.
for n in np.round(np.logspace(0,2,10)).astype(int):
    for thresh in np.linspace(0,1,10):
        print 'n={:d} thresh={:1.2e}'.format(n,thresh),
        bmap1 = (np.random.rand(n,n)>thresh).astype(float)
        bmap2 = (np.random.rand(n,n)>thresh).astype(float)
        t = time.time()
        cost,match1,match2 = matchEdgeMaps2D(bmap1,bmap2,.005,100)
        print '{:1.2e}s'.format(time.time() - t),
        npix = len(bmap1.nonzero()[0]) + len(bmap2.nonzero()[0])
        ncor = len(match1.nonzero()[0]) + len(match1.nonzero()[0])
        nout = len(((bmap1>0)&(match1==0)).nonzero()[0]) + len(((bmap2>0)&(match2==0)).nonzero()[0])
        print ncor,'+',nout,'=',ncor+nout,'==',npix
        if ncor+nout!=npix: raise ValueError("bug")
        
        match1f = match1.ravel()
        idx1 = match1f.astype(int)
        a1 = match1f.nonzero()[0]
        a2 = match1f[idx1.nonzero()[0]].ravel()
        a = vstack((a1,a2)).T

        match2f = match2.ravel()
        idx2 = match2f.astype(int)
        b1 = match2f.nonzero()[0]
        b2 = match2f[idx2.nonzero()[0]].ravel()
        b = vstack((b2,b1)).T
        b = b[b[:,0].argsort(),]
        if np.sum(a-b): raise ValueError("bug")

n=1 thresh=0.00e+00 5.89e-05s 0 + 2 = 2 == 2
n=1 thresh=1.11e-01 3.19e-05s 0 + 2 = 2 == 2
n=1 thresh=2.22e-01 2.91e-05s 0 + 2 = 2 == 2
n=1 thresh=3.33e-01 1.69e-05s 0 + 1 = 1 == 1
n=1 thresh=4.44e-01 1.60e-05s 0 + 1 = 1 == 1
n=1 thresh=5.56e-01 1.62e-05s 0 + 1 = 1 == 1
n=1 thresh=6.67e-01 1.50e-05s 0 + 1 = 1 == 1
n=1 thresh=7.78e-01 2.81e-05s 0 + 2 = 2 == 2
n=1 thresh=8.89e-01 2.31e-05s 0 + 0 = 0 == 0
n=1 thresh=1.00e+00 1.50e-05s 0 + 0 = 0 == 0
n=2 thresh=0.00e+00 5.60e-05s 6 + 2 = 8 == 8
n=2 thresh=1.11e-01 3.91e-05s 4 + 3 = 7 == 7
n=2 thresh=2.22e-01 3.79e-05s 4 + 3 = 7 == 7
n=2 thresh=3.33e-01 2.81e-05s 2 + 3 = 5 == 5
n=2 thresh=4.44e-01 1.41e-05s 0 + 3 = 3 == 3
n=2 thresh=5.56e-01 2.69e-05s 2 + 0 = 2 == 2
n=2 thresh=6.67e-01 1.41e-05s 0 + 3 = 3 == 3
n=2 thresh=7.78e-01 1.41e-05s 0 + 1 = 1 == 1
n=2 thresh=8.89e-01 2.72e-05s 2 + 0 = 2 == 2
n=2 thresh=1.00e+00 1.38e-05s 0 + 0 = 0 == 0
n=3 thresh=0.00e+00 1.59e-04s 16 + 2 = 18 == 18
n=3 thresh=1.11e-01 1.34e-04s 14 + 3 = 17 == 17
n=3 

In [8]:
# Unit test for correspondVoxels
for n in np.round(np.logspace(0,6,4,base=2)).astype(int):
    for thresh in np.linspace(.5,.9,4):
        print 'n={:d} thresh={:1.2e}'.format(n,thresh),
        bmap1 = (np.random.rand(n,n,n)>thresh).astype(float)
        bmap2 = (np.random.rand(n,n,n)>thresh).astype(float)

        t = time.time()
        cost,oc,match1,match2 = correspondVoxels(bmap1,bmap2,.005)
        print '{:1.4f}s'.format(time.time() - t),
        npix = len(bmap1.nonzero()[0]) + len(bmap2.nonzero()[0])
        ncor = len(match1.nonzero()[0]) + len(match1.nonzero()[0])
        nout = len(((bmap1>0)&(match1==0)).nonzero()[0]) + len(((bmap2>0)&(match2==0)).nonzero()[0])
        print ncor,'+',nout,'=',ncor+nout,'==',npix
        if ncor+nout!=npix: raise ValueError("bug")
        
        match1f = match1.ravel()
        idx1 = match1f.astype(int)
        a1 = match1f.nonzero()[0]
        a2 = match1f[idx1.nonzero()[0]].ravel()
        a = vstack((a1,a2)).T

        match2f = match2.ravel()
        idx2 = match2f.astype(int)
        b1 = match2f.nonzero()[0]
        b2 = match2f[idx2.nonzero()[0]].ravel()
        b = vstack((b2,b1)).T
        b = b[b[:,0].argsort(),]
        if np.sum(a-b): raise ValueError("bug")

 n=1 thresh=5.00e-01 0.0000s 0 + 0 = 0 == 0
n=1 thresh=6.33e-01 0.0000s 0 + 1 = 1 == 1
n=1 thresh=7.67e-01 0.0000s 0 + 0 = 0 == 0
n=1 thresh=9.00e-01 0.0000s 0 + 1 = 1 == 1
n=4 thresh=5.00e-01 0.0003s 32 + 33 = 65 == 65
n=4 thresh=6.33e-01 0.0002s 18 + 28 = 46 == 46
n=4 thresh=7.67e-01 0.0001s 10 + 19 = 29 == 29
n=4 thresh=9.00e-01 0.0000s 0 + 10 = 10 == 10
n=16 thresh=5.00e-01 0.0333s 2046 + 2012 = 4058 == 4058
n=16 thresh=6.33e-01 0.0167s 1074 + 1901 = 2975 == 2975
n=16 thresh=7.67e-01 0.0087s 510 + 1445 = 1955 == 1955
n=16 thresh=9.00e-01 0.0010s 74 + 748 = 822 == 822
n=64 thresh=5.00e-01 7.5874s 131232 + 131156 = 262388 == 262388
n=64 thresh=6.33e-01 3.3644s 70302 + 122002 = 192304 == 192304
n=64 thresh=7.67e-01 0.9500s 28208 + 93970 = 122178 == 122178
n=64 thresh=9.00e-01 0.1341s 5240 + 47151 = 52391 == 52391


In [9]:
# Unit test for matchEdgeMaps3D
for n in np.round(np.logspace(0,6,4,base=2)).astype(int):
    for thresh in np.linspace(.5,.9,4):
        print 'n={:d} thresh={:1.2e}'.format(n,thresh),
        bmap1 = (np.random.rand(n,n,n)>thresh).astype(float)
        bmap2 = (np.random.rand(n,n,n)>thresh).astype(float)

        t = time.time()
        cost,match1,match2 = matchEdgeMaps3D(bmap1,bmap2,.005*np.sqrt(n**3),100)
        print '{:1.4f}s'.format(time.time() - t),
        npix = len(bmap1.nonzero()[0]) + len(bmap2.nonzero()[0])
        ncor = len(match1.nonzero()[0]) + len(match1.nonzero()[0])
        nout = len(((bmap1>0)&(match1==0)).nonzero()[0]) + len(((bmap2>0)&(match2==0)).nonzero()[0])
        print ncor,'+',nout,'=',ncor+nout,'==',npix
        if ncor+nout!=npix: raise ValueError("bug")
        
        match1f = match1.ravel()
        idx1 = match1f.astype(int)
        a1 = match1f.nonzero()[0]
        a2 = match1f[idx1.nonzero()[0]].ravel()
        a = vstack((a1,a2)).T

        match2f = match2.ravel()
        idx2 = match2f.astype(int)
        b1 = match2f.nonzero()[0]
        b2 = match2f[idx2.nonzero()[0]].ravel()
        b = vstack((b2,b1)).T
        b = b[b[:,0].argsort(),]
        if np.sum(a-b): raise ValueError("bug")

n=1 thresh=5.00e-01 0.0000s 0 + 1 = 1 == 1
n=1 thresh=6.33e-01 0.0000s 0 + 1 = 1 == 1
n=1 thresh=7.67e-01 0.0000s 0 + 0 = 0 == 0
n=1 thresh=9.00e-01 0.0000s 0 + 0 = 0 == 0
n=4 thresh=5.00e-01 0.0002s 28 + 29 = 57 == 57
n=4 thresh=6.33e-01 0.0002s 16 + 25 = 41 == 41
n=4 thresh=7.67e-01 0.0001s 10 + 26 = 36 == 36
n=4 thresh=9.00e-01 0.0000s 0 + 12 = 12 == 12
n=16 thresh=5.00e-01 0.0392s 2040 + 2040 = 4080 == 4080
n=16 thresh=6.33e-01 0.0150s 1046 + 1943 = 2989 == 2989
n=16 thresh=7.67e-01 0.0059s 478 + 1440 = 1918 == 1918
n=16 thresh=9.00e-01 0.0009s 56 + 759 = 815 == 815
n=64 thresh=5.00e-01 26.4297s 130104 + 131787 = 261891 == 261891
n=64 thresh=6.33e-01 19.9401s 70558 + 121807 = 192365 == 192365
n=64 thresh=7.67e-01 18.2569s 28796 + 93349 = 122145 == 122145
n=64 thresh=9.00e-01 8.3589s 5156 + 47129 = 52285 == 52285


n=10 thresh=5.00e-01 6.77e-04s 98 48 50
